# Getting Data Regarding the Last 50 Songs Played by User on Spotify

<p>Importing required libraries</p>

In [1]:
import spotipy
import os
import json
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv

<p>Gaining access to the .env file to get the required id, secret and url</p>

In [2]:
load_dotenv()
spotipy_client_id = os.getenv('SPOTIPY_CLIENT_ID')
spotipy_client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
spotipy_redirect_url = os.getenv('SPOTIPY_REDIRECT_URL')

True

<p>Specifying type of scope required for current use.</p>
<p>Using SpotifyOAuth to autheticate requests</p>

In [4]:
scope = "user-read-recently-played"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotipy_client_id, client_secret=spotipy_client_secret, redirect_uri=spotipy_redirect_url, scope=scope))

In [5]:
results = sp.current_user_recently_played(limit=50) # getting last 50 songs played by the user

In [13]:
print(results.keys())

dict_keys(['items', 'next', 'cursors', 'limit', 'href'])


In [7]:
print(results["items"][0]) #Using this to pinpoint various features that will be added to the dataset

{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5MouCg6ta7zAxsfMEbc1uh'}, 'href': 'https://api.spotify.com/v1/artists/5MouCg6ta7zAxsfMEbc1uh', 'id': '5MouCg6ta7zAxsfMEbc1uh', 'name': 'OMI', 'type': 'artist', 'uri': 'spotify:artist:5MouCg6ta7zAxsfMEbc1uh'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', '

In [8]:
tracks_list = list() #initialising a list that will store a few details about the track

<p>Iterating through the <strong>results["items"]</strong> to get the following details</p>

 - Track Name
 - Track Link
 - Track URI
 - Artist Name 
 - Artist Link
 - Artist URI
 - Artist Popularity
 - Artist Genres
 - Track Popularity

<p>These features are then appended to the <strong>tracks_list</strong></p>

In [9]:
for idx, item in enumerate(results["items"]):
    track_name = item["track"]["name"]
    track_link = item["track"]["external_urls"]["spotify"]
    if track_link is not None:
        track_URI = track_link.split("/")[-1].split("?")[0]
    artist_name = item["track"]["artists"][0]["name"]
    artist_link = item["track"]["artists"][0]["external_urls"]["spotify"]
    if artist_link is not None:
        artist_URI = artist_link.split("/")[-1].split("?")[0]
    if artist_URI is not None:
        artist_info = sp.artist(artist_URI)
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    track_pop = item["track"]["popularity"]
    tracks_list.append({"track_uri": track_URI, "track_name": track_name, "track_pop": track_pop, 
                        "artist_uri": artist_URI, "artist_name": artist_name, "artist_pop": artist_pop, 
                        "artist_genres": artist_genres})
print(len(tracks_list))

50


In [14]:
tracks_list #displaying tracks list

[{'track_uri': '7vFoFDWqTX0mHzLfrF1Cfy',
  'track_name': 'Cheerleader - Felix Jaehn Remix Radio Edit',
  'track_pop': 79,
  'artist_uri': '5MouCg6ta7zAxsfMEbc1uh',
  'artist_name': 'OMI',
  'artist_pop': 64,
  'artist_genres': ['dance pop']},
 {'track_uri': '6dGnYIeXmHdcikdzNNDMm2',
  'track_name': 'Here Comes The Sun - Remastered 2009',
  'track_pop': 82,
  'artist_uri': '3WrFJ7ztbogyGnTHbHJFl2',
  'artist_name': 'The Beatles',
  'artist_pop': 82,
  'artist_genres': ['beatlesque',
   'british invasion',
   'classic rock',
   'merseybeat',
   'psychedelic rock',
   'rock']},
 {'track_uri': '2VxeLyX666F8uXCJ0dZF8B',
  'track_name': 'Shallow',
  'track_pop': 83,
  'artist_uri': '1HY2Jd0NmPuamShAr6KMms',
  'artist_name': 'Lady Gaga',
  'artist_pop': 83,
  'artist_genres': ['art pop', 'dance pop', 'pop']},
 {'track_uri': '7DnAm9FOTWE3cUvso43HhI',
  'track_name': 'Sweet but Psycho',
  'track_pop': 81,
  'artist_uri': '4npEfmQ6YuiwW1GpUmaq3F',
  'artist_name': 'Ava Max',
  'artist_pop': 80,


<p>Initialised features list that will contain the final features and details that will be used in EDA and to build the recommender system. We use the track URI from tracks_list and spotipy to get the features of the song.</p>
<p>We append the following features to the features_list</p>

 - Track Name
 - Artist Name
 - Artist Popularity
 - Artist Genres
 - Danceability
 - Energy
 - Loudness
 - Speechiness
 - Acousticness
 - Instrumentalness
 - Liveness
 - Valence
 - Tempo
 - Duration (in ms)
 - Time Signature

In [11]:
features_list = list()
for track in tracks_list:
    try:
        x = sp.audio_features(track["track_uri"])
        features_list.append({"track_name":track["track_name"], "artist_name":track["artist_name"], 
                                "artist_pop":track["artist_pop"], "artist_genres":track["artist_genres"],
                                "danceability":x[0]["danceability"], "energy":x[0]["energy"], "loudness":x[0]["loudness"], 
                                "speechiness":x[0]["speechiness"], "acousticness":x[0]["acousticness"], 
                                "instrumentalness":x[0]["instrumentalness"], "liveness":x[0]["liveness"], 
                                "valence":x[0]["valence"], "tempo":x[0]["tempo"], 
                                "duration_ms":x[0]["duration_ms"], "time_signature":x[0]["time_signature"],
                                "track_pop":track["track_pop"]})
    except:
        print("Error")
    

In [12]:
print(len(features_list))

50
